<h3>CA675 Cloud Assignment</h3><br>
Name - Sarthak Bhagwat Taru<br> 
StudentID - 21261303<br>
Email - sarthak.taru2@mail.dcu.ie

<br><br><b>Task 1 - Acquire the top 2,00,000 posts by ViewCount</b><br>
Query executed on stackexchange data platform to retrieve the top 2,00,000 posts by viewcount
https://data.stackexchange.com/stackoverflow/query/new

To retrieve top 2,00,000 records by viewcount, I have first find out the max viewcount from posts table and which was 10062790.<br>
Query - select max(ViewCount) as viewcount from posts;<br>
With that 4 queries, the sql is executed in following manner to extract top 2,00,000 records.<br>
Query 1 - select top 50000 psts.Id,psts.Score,psts.ViewCount,psts.Body,
users.Id as OwnerUserId,
users.DisplayName,psts.Title,psts.Tags
from Posts psts inner join Users users 
on psts.OwnerUserId=users.Id
where psts.ViewCount>1000 and 
psts.ViewCount < 10062791
order by psts.ViewCount desc
<br><br>
Query 2 - select top 50000 psts.Id,psts.Score,psts.ViewCount,psts.Body,
users.Id as OwnerUserId,
users.DisplayName,psts.Title,psts.Tags
from Posts psts inner join Users users 
on psts.OwnerUserId=users.Id
where psts.ViewCount>1000 and 
psts.ViewCount < 124974
order by psts.ViewCount desc
<br><br>
Query 3 - select top 50000 psts.Id,psts.Score,psts.ViewCount,psts.Body,
users.Id as OwnerUserId,
users.DisplayName,psts.Title,psts.Tags
from Posts psts inner join Users users 
on psts.OwnerUserId=users.Id
where psts.ViewCount>1000 and 
psts.ViewCount < 73139
order by psts.ViewCount desc
<br><br>
Query 4 - select top 50000 psts.Id,psts.Score,psts.ViewCount,psts.Body,
users.Id as OwnerUserId,
users.DisplayName,psts.Title,psts.Tags
from Posts psts inner join Users users 
on psts.OwnerUserId=users.Id
where psts.ViewCount>1000 and 
psts.ViewCount < 52110
order by psts.ViewCount desc
<br><br>
In output, the 4 csv files were created and combined all that in one combined.csv





<b>Task 2 & 3 - Use Pig/Hive/MapReduce - Extract, Transform and Load the data as applicable to get:</b><br>
This assignment is completed on Google Cloud Platform by creating the cluster with Dataproc service. The google cloud was easy to understand and the cluster creation was easy through DataProc. Instance was configured with Jupyter notebook to execute the python code. I have used the Hive in cluster creation because of the prior hands-on experience in sql query language.<br>

Created the ca675 Database and Table under same database.<br>
Table creation query - CREATE TABLE IF NOT EXISTS ca675.Stackexchange (Id int,Score int,ViewCount int,Body string, OwnerUserId int, OwnerDisplayName string, Title string, Tags string)  ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde';<br>



<br><b>Data Cleaning</b><br>
Before loading data into the Hive, I have cleaned the body column on google colab jupyter notebook with following code snippet.<br>

df['Body'] = df['Body'].str.replace(r'<[^<>]*>', '', regex=True)<br>
df["Body"] = df['Body'].str.replace('[^\w\s]','', regex=True)<br>
df["Body"] = df['Body'].str.replace('[^a-zA-Z]', ' ', regex=True)<br>
df["Body"] = df['Body'].str.replace(r'\n',' ', regex=True)<br><br>

<br><b>Data Loading into Hive</b><br>
Uploaded the csv file in assigned google cloud storage and loaded the data
through below sql query - <br>LOAD DATA INPATH ‘gs://dataproc-staging-us-central1-1078341724532-
t5a0gorr/combined.csv’ INTO TABLE ca675.Stackexchange;<br><br>

After Succesful data loading, the Jupyter is accessed by cloud public address.<br>
Installed below packages in cloud ubantu environment -<br>
! pip install sasl<br>
! pip install thrift<br>
! pip install thrift-sasl<br>
! pip install PyHive<br>





In [20]:
from pyhive import hive
from tabulate import tabulate
import pandas as pd

In [21]:
host_name = "localhost"
port = 10000
user = "sarthak_taru2"
password = "509485636803468691"
database="ca675"
conn = hive.Connection(host=host_name,port=port,username=user,password=password,database=database,auth='CUSTOM')
cur = conn.cursor()

<br><b>Task 2.1. The top 10 posts by score</b><br>

In [30]:
cur.execute('''select Id,OwnerUserId,OwnerDisplayName,Score,ViewCount
FROM ca675.stackexchanges order by Score desc LIMIT 10''')
result = cur.fetchall()
df = pd.DataFrame (result, columns =['Id','OwnerUserId','OwnerDisplayName','Score','ViewCount'])
display(df)

,Id,OwnerUserId,OwnerDisplayName,Score,ViewCount
0,11227809,87234,GManNickG,25933,1649855
1,927358,89904,Hamza Yerlikaya,23348,10062790
2,2003505,95592,Matthew Rankin,18514,9285139
3,292357,6068,pupeno,12834,3041604
4,231767,18300,Alex. S.,11551,2681330
5,477816,12870,Oli,10921,3269028
6,348170,14069,paxos1977,10079,3985243
7,5767325,364969,Walker,9931,8937271
8,6591213,338204,Forrest,9792,3729583
9,1642028,87234,GManNickG,9560,877861


<br><b>Task 2.2 The top 10 users by post score</b><br>

In [28]:
cur.execute('''
select OwnerUserId,max(OwnerDisplayName) , sum(Score) as score
from stackexchanges
group by OwnerUserId
order by Score desc limit 10
''')
result = cur.fetchall()
df1 = pd.DataFrame (result, columns = ['OwnerUserId','OwnerDisplayName','score'])
display(df1)

,OwnerUserId,OwnerDisplayName,score
0,87234,GManNickG,37672
1,4883,readonly,28817
2,9951,e-satis,26878
3,6068,pupeno,25944
4,89904,Hamza Yerlikaya,24024
5,51816,Joan Venge,23763
6,49153,Ali,20203
7,179736,TIMEX,19603
8,95592,Matthew Rankin,19479
9,63051,flybywire,19362


<br><b>Task 3 - he number of distinct users, who used the word “cloud” in one of their
posts</b><br>
Here I have considered the Body <b>and</b> Title to find out the total count of owner user id who used the word cloud
in one of their posts.

In [39]:
cur.execute('''
SELECT
COUNT(DISTINCT OwnerUserId) as owner_user_count
FROM stackexchanges
WHERE Title LIKE '% Cloud %' AND Body LIKE '% Cloud %'
''')
result = cur.fetchall()
df2 = pd.DataFrame (result, columns = ['Owner_User_Count'])
display(df2)

,Owner_User_Count
0,168


<br><b>Task 4 - calculate the per-user TF-IDF of the top 10 terms for each of the top 10 users</b><br> 
So first of all, I have extracted the top 10 owner user id with their titles and created the list of top 10 users.

In [35]:
df3 = pd.read_sql("""
SELECT OwnerUserId,OwnerDisplayName,Title,Body
from stackexchanges
WHERE OwnerUserId
IN
(
select OwnerUserId from(select OwnerUserId,max(OwnerDisplayName),sum(Score) as score
from stackexchanges
group by OwnerUserId
order by score desc limit 10)stack
)
order by OwnerUserId""", conn)
result = cur.fetchall()
top_10_users = list(df3["owneruserid"].unique())
display(top_10_users)

[4883, 6068, 9951, 49153, 51816, 63051, 87234, 89904, 95592, 179736]

I have taken the reference from below site to implement tfidf<br> https://scikitlearn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html<br>
for each of the owner and their top 10 words plot the table accordingly.

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

def calculate_tfidf(title_each_user):
    TitleCopy = title_each_user["title"].copy()
    vectorizer = TfidfVectorizer(stop_words='english',lowercase=True)
    response = vectorizer.fit_transform(title_each_user['body'].str.cat(' '+TitleCopy))
    df_tfidf_words = pd.DataFrame(response.toarray(),columns=vectorizer.get_feature_names()) #calculating tf-idf values
    df_final_result_per_user=df_tfidf_words.sum(axis=0, numeric_only= True) #suming up the tf-idf to get the top 10 words
    top_words = df_final_result_per_user.nlargest(n=10)
    top_10_words = list(top_words.index)
    return(df_tfidf_words[top_10_words])


for each_item in top_10_users: # Iterating through top 10 users
    owneruserid = str(each_item)
    selectTitle=df3.loc[df3['owneruserid'] == each_item]
    selectTitle.insert(0,'Owneruserid',each_item)
    tfidf=calculate_tfidf(selectTitle)
    tfidf.insert(0,'owneruserid',owneruserid)
    display(tfidf)

,owneruserid,python,use,ruby,list,table,way,difference,branch,method,process
0,4883,0.000000,0.000000,0.000000,0.000000,0.521156,0.000000,0.000000,0.000000,0.000000,0.000000
1,4883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,4883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4883,0.072578,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.099463,0.000000
4,4883,0.160147,0.000000,0.219470,0.438941,0.000000,0.184107,0.000000,0.000000,0.219470,0.000000
5,4883,0.000000,0.000000,0.000000,0.202044,0.000000,0.169489,0.000000,0.000000,0.000000,0.000000
6,4883,0.000000,0.198917,0.000000,0.000000,0.255265,0.099459,0.000000,0.000000,0.000000,0.382898
7,4883,0.121700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,4883,0.000000,0.000000,0.000000,0.000000,0.000000,0.181739,0.000000,0.000000,0.000000,0.000000
9,4883,0.219948,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.401900,0.000000


,owneruserid,file,java,android,git,sql,way,like,time,code,tables
0,6068,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,6068,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.198534,0.000000,0.000000
2,6068,0.000000,0.000000,0.000000,0.728914,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,6068,0.000000,0.000000,0.130150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,6068,0.000000,0.000000,0.173267,0.000000,0.000000,0.000000,0.000000,0.389852,0.000000,0.000000
5,6068,0.000000,0.000000,0.000000,0.000000,0.000000,0.140540,0.000000,0.000000,0.000000,0.000000
6,6068,0.000000,0.386646,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,6068,0.000000,0.000000,0.275010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,6068,0.000000,0.000000,0.000000,0.000000,0.217416,0.000000,0.000000,0.000000,0.000000,0.233437
9,6068,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047141,0.000000,0.000000,0.000000


,owneruserid,python,like,git,head,does,using,way,use,know,string
0,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.299606,0.000000,0.000000,0.000000,0.000000
1,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,9951,0.000000,0.000000,0.179128,0.734307,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,9951,0.000000,0.000000,0.187925,0.000000,0.062642,0.000000,0.059024,0.000000,0.000000,0.000000
5,9951,0.109076,0.092250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,9951,0.000000,0.033760,0.000000,0.000000,0.000000,0.159673,0.112839,0.119755,0.142318,0.000000
7,9951,0.000000,0.000000,0.338695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,9951,0.081895,0.138523,0.000000,0.000000,0.163789,0.000000,0.000000,0.000000,0.072993,0.000000
9,9951,0.000000,0.000000,0.416684,0.379585,0.138895,0.000000,0.000000,0.138895,0.000000,0.000000


,owneruserid,javascript,array,using,php,jquery,file,want,gt,class,string
0,49153,0.000000,0.000000,0.076582,0.000000,0.0,0.000000,0.095828,0.000000,0.000000,0.205167
1,49153,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,49153,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.049360,0.000000
3,49153,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.084953,0.000000,0.000000,0.000000
4,49153,0.000000,0.524079,0.053427,0.032054,0.0,0.000000,0.033427,0.452884,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
74,49153,0.000000,0.000000,0.000000,0.000000,0.0,0.170775,0.000000,0.000000,0.085388,0.000000
75,49153,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
76,49153,0.085136,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
77,49153,0.000000,0.000000,0.067637,0.000000,0.0,0.000000,0.169268,0.000000,0.000000,0.000000


,owneruserid,python,like,list,string,want,class,index,value,use,function
0,51816,0.000000,0.075548,0.000000,0.000000,0.000000,0.000000,0.0,0.126344,0.000000,0.0
1,51816,0.000000,0.082816,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,51816,0.000000,0.000000,0.143562,0.034883,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,51816,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,51816,0.000000,0.000000,0.000000,0.000000,0.000000,0.033316,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
61,51816,0.000000,0.000000,0.000000,0.000000,0.000000,0.350645,0.0,0.000000,0.164888,0.0
62,51816,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
63,51816,0.393731,0.193425,0.000000,0.295685,0.266721,0.000000,0.0,0.000000,0.000000,0.0
64,51816,0.231039,0.075667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.238024,0.0


,owneruserid,file,python,want,vs,use,standard,command,bash,java,output
0,63051,0.000000,0.328540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,63051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,63051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,63051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,63051,0.000000,0.000000,0.068103,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,63051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,63051,0.192038,0.000000,0.089639,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,63051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,63051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.157276,0.000000
9,63051,0.000000,0.146353,0.000000,0.153253,0.306505,0.000000,0.000000,0.000000,0.000000,0.000000


,owneruserid,idiom,int,copy,array,arraysize,data,code,does,mentioned,places
0,87234,0.480997,0.000000,0.360748,0.091453,0.00000,0.00000,0.000000,0.240499,0.240499,0.240499
1,87234,0.000000,0.203351,0.000000,0.261452,0.30558,0.30558,0.145251,0.000000,0.000000,0.000000
2,87234,0.000000,0.267377,0.000000,0.000000,0.00000,0.00000,0.133688,0.000000,0.000000,0.000000


,owneruserid,file,timer,new,java,setpreferredwidth,tablegetcolumnmodelgetcolumn,files,application,jar,array
0,89904,0.000000,0.0000,0.000000,0.040077,0.606105,0.606105,0.000000,0.000000,0.000000,0.000000
1,89904,0.351468,0.0000,0.000000,0.000000,0.000000,0.000000,0.312480,0.000000,0.000000,0.000000
2,89904,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.085641,0.000000,0.000000,0.000000
3,89904,0.463315,0.0000,0.000000,0.000000,0.000000,0.000000,0.102980,0.264787,0.264787,0.000000
4,89904,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,89904,0.000000,0.0000,0.000000,0.313903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,89904,0.000000,0.6912,0.235033,0.091408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,89904,0.075479,0.0000,0.431366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,89904,0.000000,0.0000,0.000000,0.163094,0.000000,0.000000,0.000000,0.000000,0.000000,0.493305
9,89904,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.097851,0.251599,0.251599,0.000000


,owneruserid,install,python,pip,branch,installed,flask,version,started,navigation,way
0,95592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062210
1,95592,0.000000,0.373806,0.000000,0.000000,0.000000,0.000000,0.000000,0.516883,0.000000,0.129146
2,95592,0.427251,0.000000,0.284834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.098407
3,95592,0.000000,0.000000,0.000000,0.000000,0.436263,0.520551,0.520551,0.000000,0.000000,0.130063
4,95592,0.262228,0.104891,0.104891,0.000000,0.121554,0.000000,0.000000,0.000000,0.000000,0.000000
5,95592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.495445,0.014563
6,95592,0.466707,0.262523,0.320861,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060466
7,95592,0.000000,0.000000,0.000000,0.697882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,owneruserid,python,want,string,user,dictionary,file,return,use,way,print
0,179736,0.170174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.206089,0.000000,0.000000
1,179736,0.122064,0.000000,0.163895,0.000000,0.172007,0.000000,0.000000,0.000000,0.000000,0.000000
2,179736,0.000000,0.097942,0.000000,0.302904,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,179736,0.000000,0.000000,0.000000,0.000000,0.000000,0.272653,0.000000,0.000000,0.000000,0.125299
4,179736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.179267
...,...,...,...,...,...,...,...,...,...,...,...
110,179736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.093065,0.000000,0.000000
111,179736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.116232,0.126013,0.000000
112,179736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.196130,0.000000
113,179736,0.000000,0.140351,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
